<a href="https://colab.research.google.com/github/dolmarawat/NLP-332-Final-Project/blob/main/Data_acquistionscrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas requests

In [2]:
import requests
import pandas as pd
import datetime as dt


In [3]:
def scrape_thread_json(thread_url: str) -> pd.DataFrame:
    """
    Scrape a single Reddit thread (post + all comments) using the public .json endpoint.
    Does NOT require Reddit API credentials.
    """

    # Ensure we use the .json endpoint
    if not thread_url.endswith(".json"):
        if thread_url.endswith("/"):
            thread_url = thread_url + ".json"
        else:
            thread_url = thread_url + "/.json"

    headers = {"User-Agent": "Mozilla/5.0 (compatible; IST322-NLP-Project/1.0)"}

    resp = requests.get(thread_url, headers=headers)
    resp.raise_for_status()

    data = resp.json()
    records = []

    # ---- 1. Main post ----
    post = data[0]["data"]["children"][0]["data"]

    post_id = post.get("id")
    records.append({
        "id": post_id,
        "type": "post",
        "parent_post_id": None,
        "title": post.get("title", ""),
        "text": post.get("selftext", ""),
        "score": post.get("score", 0),
        "author": post.get("author", None),
        "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0)),
        "url": "https://www.reddit.com" + post.get("permalink", "")
    })

    # ---- 2. Comments (recursive) ----
    comments = data[1]["data"]["children"]

    def extract_comment_tree(comment_list):
        for c in comment_list:
            if c["kind"] != "t1":  # t1 = comment
                continue
            comment = c["data"]

            records.append({
                "id": comment.get("id"),
                "type": "comment",
                "parent_post_id": post_id,
                "title": None,
                "text": comment.get("body", ""),
                "score": comment.get("score", 0),
                "author": comment.get("author", None),
                "timestamp": dt.datetime.fromtimestamp(comment.get("created_utc", 0)),
                "url": "https://www.reddit.com" + comment.get("permalink", "")
            })

            # Recurse replies
            replies = comment.get("replies")
            if replies and isinstance(replies, dict):
                child_comments = replies["data"]["children"]
                extract_comment_tree(child_comments)

    extract_comment_tree(comments)

    df = pd.DataFrame(records)
    return df


In [4]:
url = "https://www.reddit.com/r/Diamonds/comments/10jf41v/why_do_people_still_choose_natural_over_lab/"

df = scrape_thread_json(url)
print("Total records scraped:", len(df))
df.head()

HTTPError: 403 Client Error: Blocked for url: https://www.reddit.com/r/Diamonds/comments/10jf41v/why_do_people_still_choose_natural_over_lab/.json

In [5]:
import requests

test_url = "https://www.reddit.com/r/Diamonds/comments/10jf41v/why_do_people_still_choose_natural_over_lab/.json"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

resp = requests.get(test_url, headers=headers)

print("Status code:", resp.status_code)
print(resp.text[:500])

Status code: 403
<body class=theme-beta><div><style>.theme-light,:root{--rem360:22.5rem;--rem320:20rem;--rem192:12rem;--rem144:9rem;--rem128:8rem;--rem96:6rem;--rem90:5.625rem;--rem88:5.5rem;--rem64:4rem;--rem56:3.5rem;--rem48:3rem;--rem40:2.5rem;--rem36:2.25rem;--rem32:2rem;--rem28:1.75rem;--rem26:1.625rem;--rem24:1.5rem;--rem22:1.375rem;--rem20:1.25rem;--rem18:1.125rem;--rem16:1rem;--rem15:0.9375rem;--rem14:0.875rem;--rem12:0.75rem;--rem10:0.625rem;--rem8:0.5rem;--rem6:0.375rem;--rem4:0.25rem;--rem2:0.125rem;-


In [6]:
import requests
import pandas as pd
import datetime as dt

def scrape_thread_json(thread_url: str) -> pd.DataFrame:
    """
    Scrape a single Reddit thread (post + all comments) using Reddit's public JSON API.
    Uses api.reddit.com and raw_json=1 to reduce blocking.
    """

    # Convert standard URL → API URL
    # e.g. https://www.reddit.com/r/... → https://api.reddit.com/r/...
    api_url = thread_url
    api_url = api_url.replace("https://www.reddit.com", "https://api.reddit.com")
    api_url = api_url.replace("http://www.reddit.com", "https://api.reddit.com")

    if not api_url.endswith("/"):
        api_url = api_url + "/"

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/120.0 Safari/537.36",
        "Accept": "application/json, text/plain, */*"
    }

    params = {"raw_json": 1}

    resp = requests.get(api_url, headers=headers, params=params)
    print("Status code:", resp.status_code)
    resp.raise_for_status()

    data = resp.json()
    records = []

    # ---- 1. Main post ----
    post = data[0]["data"]["children"][0]["data"]
    post_id = post.get("id")

    records.append({
        "id": post_id,
        "type": "post",
        "parent_post_id": None,
        "title": post.get("title", ""),
        "text": post.get("selftext", ""),
        "score": post.get("score", 0),
        "author": post.get("author", None),
        "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0)),
        "url": "https://www.reddit.com" + post.get("permalink", "")
    })

    # ---- 2. Comments (tree) ----
    comments = data[1]["data"]["children"]

    def extract_comment_tree(comment_list):
        for c in comment_list:
            if c["kind"] != "t1":  # t1 = comment
                continue
            comment = c["data"]

            records.append({
                "id": comment.get("id"),
                "type": "comment",
                "parent_post_id": post_id,
                "title": None,
                "text": comment.get("body", ""),
                "score": comment.get("score", 0),
                "author": comment.get("author", None),
                "timestamp": dt.datetime.fromtimestamp(comment.get("created_utc", 0)),
                "url": "https://www.reddit.com" + comment.get("permalink", "")
            })

            replies = comment.get("replies")
            if replies and isinstance(replies, dict):
                child_comments = replies["data"]["children"]
                extract_comment_tree(child_comments)

    extract_comment_tree(comments)

    df = pd.DataFrame(records)
    return df


In [7]:
url = "https://www.reddit.com/r/Diamonds/comments/10jf41v/why_do_people_still_choose_natural_over_lab/"

df = scrape_thread_json(url)
print("Rows:", len(df))
df.head()

Status code: 403


HTTPError: 403 Client Error: Blocked for url: https://api.reddit.com/r/Diamonds/comments/10jf41v/why_do_people_still_choose_natural_over_lab/?raw_json=1

In [17]:
import json
import pandas as pd
import datetime as dt

with open("Diamonds.1.json", "r") as f:
    data = json.load(f)

records = []

post = data[0]["data"]["children"][0]["data"]
post_id = post.get("id")

records.append({
    "id": post_id,
    "type": "post",
    "parent_post_id": None,
    "title": post.get("title", ""),
    "text": post.get("selftext", ""),
    "score": post.get("score", 0),
    "author": post.get("author", None),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0)),
    "url": "https://www.reddit.com" + post.get("permalink", "")
})

comments = data[1]["data"]["children"]

def extract_comment_tree(comment_list):
    for c in comment_list:
        if c["kind"] != "t1":
            continue
        comment = c["data"]
        records.append({
            "id": comment.get("id"),
            "type": "comment",
            "parent_post_id": post_id,
            "title": None,
            "text": comment.get("body", ""),
            "score": comment.get("score", 0),
            "author": comment.get("author", None),
            "timestamp": dt.datetime.fromtimestamp(comment.get("created_utc", 0)),
            "url": "https://www.reddit.com" + comment.get("permalink", "")
        })
        replies = comment.get("replies")
        if replies and isinstance(replies, dict):
            child_comments = replies["data"]["children"]
            extract_comment_tree(child_comments)

extract_comment_tree(comments)

df = pd.DataFrame(records)
print("Rows:", len(df))
df.head()

Rows: 193


,id,type,parent_post_id,title,text,score,author,timestamp,url
0,10jf41v,post,None,Why do people still choose natural over lab di...,Lab made diamonds are becoming more popular an...,138,Alyssa_joy_11339,2023-01-23 15:28:32,https://www.reddit.com/r/Diamonds/comments/10j...
1,j5lb5tv,comment,10jf41v,None,Keep the comments civil. You can have an opini...,1,lucerndia,2023-01-23 20:36:12,https://www.reddit.com/r/Diamonds/comments/10j...
2,j5kiii4,comment,10jf41v,None,Many people still view lab diamonds as fakes/C...,110,BeachPlze,2023-01-23 17:37:58,https://www.reddit.com/r/Diamonds/comments/10j...
3,j5kz8qz,comment,10jf41v,None,I spend a good part of my job explaining that ...,68,rosiehideshere,2023-01-23 19:21:38,https://www.reddit.com/r/Diamonds/comments/10j...
4,j5l31dv,comment,10jf41v,None,"Meh, everyone is going to have their own opini...",21,BeachPlze,2023-01-23 19:45:36,https://www.reddit.com/r/Diamonds/comments/10j...


In [25]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("diamonds_thread.json.js", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)

comments_df = df[df["type"] == "comment"][["author", "text"]]
print("Number of comments:", len(comments_df))
comments_df.head()


Number of comments: 192


,author,text
1,lucerndia,Keep the comments civil. You can have an opini...
2,BeachPlze,Many people still view lab diamonds as fakes/C...
3,rosiehideshere,I spend a good part of my job explaining that ...
4,BeachPlze,"Meh, everyone is going to have their own opini..."
5,TheCaliforniaOp,Yes! I’m going to remember the tomato analogy.


In [26]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,Alyssa_joy_11339,Lab made diamonds are becoming more popular an...
1,lucerndia,Keep the comments civil. You can have an opini...
2,BeachPlze,Many people still view lab diamonds as fakes/C...
3,rosiehideshere,I spend a good part of my job explaining that ...
4,BeachPlze,"Meh, everyone is going to have their own opini..."


In [27]:
df.to_csv("/content/drive/MyDrive/Diamondscomments1_data.csv", index=False)

In [28]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("diamonds_comments2.json.js", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)

comments_df = df[df["type"] == "comment"][["author", "text"]]
print("Number of comments:", len(comments_df))

Number of comments: 79


In [29]:
comments_df.head()

,author,text
1,JPathway_UK,Unless you are getting a custom cut stone from...
2,transat_prof,Tons of a bad reviews for Brilliant Earth. Aby...
3,andee_sings,All of these diamond vendors share the same in...
4,Miss-KJ,I have now bought several custom diamond jewel...
5,eviestephenson2011,Team Sonia vote from me as well 🥰


In [30]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,me20000,"Hey all, I'm deep in the engagement ring searc..."
1,JPathway_UK,Unless you are getting a custom cut stone from...
2,transat_prof,Tons of a bad reviews for Brilliant Earth. Aby...
3,andee_sings,All of these diamond vendors share the same in...
4,Miss-KJ,I have now bought several custom diamond jewel...


In [31]:
df.to_csv("/content/drive/MyDrive/Diamondscomments2_data.csv", index=False)

In [32]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("diamonds_comments3.json.txt", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)

comments_df = df[df["type"] == "comment"][["author", "text"]]
print("Number of comments:", len(comments_df))

Number of comments: 56


In [33]:
comments_df.head()

,author,text
1,[deleted],[removed]
2,Diamonds-ModTeam,Your post or comment was removed because it vi...
3,Au_sum_sauce,"I like them both, but here's a short answer fo..."
4,pargofan,Curious what you think of this article that sa...
5,Au_sum_sauce,This is an Abe Mor ad. I'm not going to read i...


In [34]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,daskeletor1298,
1,[deleted],[removed]
2,Diamonds-ModTeam,Your post or comment was removed because it vi...
3,Au_sum_sauce,"I like them both, but here's a short answer fo..."
4,pargofan,Curious what you think of this article that sa...


In [35]:
df.to_csv("/content/drive/MyDrive/Diamondscomments3_data.csv", index=False)

In [41]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("diamonds_comments4.json.jsn", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)

comments_df = df[df["type"] == "comment"][["author", "text"]]

FileNotFoundError: [Errno 2] No such file or directory: 'diamonds_comments4.json.jsn'

In [37]:
print("Number of comments:", len(comments_df))

Number of comments: 56


In [38]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,daskeletor1298,
1,[deleted],[removed]
2,Diamonds-ModTeam,Your post or comment was removed because it vi...
3,Au_sum_sauce,"I like them both, but here's a short answer fo..."
4,pargofan,Curious what you think of this article that sa...


In [39]:
df.to_csv("/content/drive/MyDrive/Diamondscomments4_data.csv", index=False)

In [40]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("diamonds_comments5.json.txt", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)

comments_df = df[df["type"] == "comment"][["author", "text"]]

In [42]:
print("Number of comments:", len(comments_df))

Number of comments: 126


In [44]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,Responsible-Book2625,I guess I’m wondering why lab grown diamonds a...
1,VividAd6825,There's a few reasons why. \n\nThe lab diamond...
2,Individual_Tip8728,What do you think about brilliance.com?
3,Expert_Onion6938,"Overpriced to an extent, we sell the same ring..."
4,Individual_Tip8728,Are you guys a brick and mortar store? What ar...


In [45]:
df.to_csv("/content/drive/MyDrive/Diamondscomments5_data1.csv", index=False)

In [47]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("diamonds_comments4.json", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)


In [48]:
print("Number of comments:", len(comments_df))

Number of comments: 126


In [49]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,SeriousLion2465,I am looking to purchase a pair of diamond stu...
1,sinjin_wolfe,Even if you went with natural (mined) diamonds...
2,Cold-You-4598,"I love my lab diamonds, I think some people ar..."
3,Puzzleheaded-Ad2257,"They are chemically identical, look and behave..."
4,rosemallows,"It depends, right? I prefer lab diamonds for e..."


In [50]:
df.to_csv("/content/drive/MyDrive/Diamondscomments4_data1.csv", index=False)

In [51]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("AITA_Comments1.txt", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)

comments_df = df[df["type"] == "comment"][["author", "text"]]

In [52]:
print("Number of comments:", len(comments_df))

Number of comments: 197


In [53]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,throwaway1578650,"Throwaway account, for reasons.\nI (30m) propo..."
1,Judgement_Bot_AITA,Welcome to /r/AmITheAsshole. Please view our [...
2,CZT1991,This is a pretty good sign of how the actual w...
3,throwaway1578650,"It wasn't for insurance, but we were both disc..."
4,ConsistentCheesecake,> both of us are still paying off loans from o...


In [65]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("AITA_COMMENTS2.json", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)

comments_df = df[df["type"] == "comment"][["author", "text"]]

In [66]:
print("Number of comments:", len(comments_df))

Number of comments: 192


In [67]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,MathematicianNo3110,I've been with my gf for a few years now and t...
1,AutoModerator,\nWelcome to /r/AmITheAsshole. Please view our...
2,VinnyVincinny,NTA but you must realize that diamonds are not...
3,MrVronskij,"Thats definitely true, but not being able to b..."
4,VinnyVincinny,It's a tone that's being set here. Bet he's go...


In [68]:
df.to_csv("/content/drive/MyDrive/Refusing to exchange it for a natural stone?.csv", index=False)

In [69]:
import json
import pandas as pd
import datetime as dt

# 1. Load the JSON file you saved
with open("AITA_Comments3.json", "r") as f:
    data = json.load(f)

records = []

# 2. Main post (optional – you can skip this if you only care about comments)
post = data[0]["data"]["children"][0]["data"]
post_id = post["id"]

records.append({
    "type": "post",
    "author": post.get("author", None),
    "text": post.get("selftext", ""),
    "timestamp": dt.datetime.fromtimestamp(post.get("created_utc", 0))
})

# 3. Recursively walk all comments and collect username + text
def extract_comments(children):
    for child in children:
        if child["kind"] != "t1":    # t1 = comment
            continue

        c = child["data"]
        records.append({
            "type": "comment",
            "author": c.get("author", None),
            "text": c.get("body", ""),
            "timestamp": dt.datetime.fromtimestamp(c.get("created_utc", 0))
        })

        # If there are replies, recurse
        replies = c.get("replies")
        if replies and isinstance(replies, dict):
            extract_comments(replies["data"]["children"])

# comments live in data[1]
extract_comments(data[1]["data"]["children"])

# 4. Turn into a DataFrame and filter to comments only (if you want)
df = pd.DataFrame(records)


In [70]:
print("Number of comments:", len(comments_df))

Number of comments: 192


In [71]:
df = df.rename(columns={'author': 'username', 'text': 'comments'})
df = df.drop(columns=['type', 'timestamp'])
df.head()

,username,comments
0,Atom___,I’ll keep it as short as I can. GF and I have ...
1,Citychic88,I think the bigger issue is that you didn't re...
2,easychees,"Well, the fiancee did say that she wanted to r..."
3,Citychic88,To be fair he did say that she said she wanted...
4,poopja,Okay but you shouldn't be buying a ring before...


In [72]:
df.to_csv("/content/drive/MyDrive/ Buying my GF a lab made diamond ring without consulting her?.csv", index=False)

In [74]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 6.3 MB/s eta 0:00:00


In [75]:
import praw
import pandas as pd
import datetime as dt

reddit = praw.Reddit(
    client_id="YOUR_ID",
    client_secret="YOUR_SECRET",
    user_agent="labgrown-nlp-project"
)

url = "https://www.reddit.com/r/AmItheAsshole/comments/fchplg/aita_for_buying_my_28m_gf_29f_a_lab_made_diamond/"

submission = reddit.submission(url=url)
submission.comments.replace_more(limit=None)

records = []

for comment in submission.comments.list():
    records.append({
        "author": str(comment.author),
        "text": comment.body,
        "timestamp": dt.datetime.fromtimestamp(comment.created_utc)
    })

df = pd.DataFrame(records)
print(len(df))

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



ResponseException: received 401 HTTP response

In [76]:
!pip install beautifulsoup4 requests pandas

In [77]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

def scrape_old_reddit_thread(url: str) -> pd.DataFrame:
    # make sure we hit old.reddit, not www.reddit
    url = url.replace("https://www.reddit.com", "https://old.reddit.com")
    url = url.replace("http://www.reddit.com", "https://old.reddit.com")

    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; LabGrownNLP/1.0)"
    }

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    comments = []

    # each comment is a "thing" with data-type="comment"
    for thing in soup.find_all("div", class_="thing", attrs={"data-type": "comment"}):
        author = thing.get("data-author")

        # comment text is inside the markdown body
        body_div = thing.find("div", class_="usertext-body")
        md_div = body_div.find("div", class_="md") if body_div else None
        text = md_div.get_text(" ", strip=True) if md_div else ""

        # skip deleted/removed if you want
        if not text:
            continue

        comments.append({
            "author": author,
            "text": text
        })

    return pd.DataFrame(comments)


In [79]:
url = "www.reddit.com/r/AmItheAsshole/comments/fchplg/aita_for_buying_my_28m_gf_29f_a_lab_made_diamond/"
df = scrape_old_reddit_thread(url)
print("Comments scraped:", len(df))
df.head()

MissingSchema: Invalid URL 'www.reddit.com/r/AmItheAsshole/comments/fchplg/aita_for_buying_my_28m_gf_29f_a_lab_made_diamond/': No scheme supplied. Perhaps you meant https://www.reddit.com/r/AmItheAsshole/comments/fchplg/aita_for_buying_my_28m_gf_29f_a_lab_made_diamond/?

In [80]:
from google.colab import files

uploaded = files.upload()

Saving AITA For buying my fiance a lab grown diamond and refusing to exchange it for a natural stone_ _ r_AmItheAsshole.html to AITA For buying my fiance a lab grown diamond and refusing to exchange it for a natural stone_ _ r_AmItheAsshole (1).html


In [81]:
from bs4 import BeautifulSoup
import pandas as pd

def parse_old_reddit_html(path):
    """
    Parse a saved old.reddit HTML page.
    Extracts author + comment text.
    Works best when comments are expanded before saving.
    """
    with open(path, "r", encoding="utf-8") as f:
        html = f.read()

    soup = BeautifulSoup(html, "html.parser")

    comments = []

    # Each comment is a 'thing' with data-type="comment"
    for thing in soup.find_all("div", class_="thing", attrs={"data-type": "comment"}):
        author = thing.get("data-author")

        # Skip deleted/removed
        if not author or author in ["[deleted]", "[removed]"]:
            continue

        # Extract the markdown content (comment text)
        body = thing.find("div", class_="usertext-body")
        md = body.find("div", class_="md") if body else None
        text = md.get_text(" ", strip=True) if md else ""

        if text.strip():
            comments.append({
                "author": author,
                "text": text
            })

    return pd.DataFrame(comments)


In [82]:
df = parse_old_reddit_html("AITA For buying my fiance a lab grown diamond and refusing to exchange it for a natural stone_ _ r_AmItheAsshole (1).html")
print("Total comments extracted:", len(df))
df.head()


Total comments extracted: 0


""


In [1]:
from google.colab import files
uploaded = files.upload()

Saving AITA for getting my fiancée a “fake” diamond ring_ _ AmItheAsshole.html to AITA for getting my fiancée a “fake” diamond ring_ _ AmItheAsshole.html


In [2]:
!pip install beautifulsoup4 pandas

In [3]:
from bs4 import BeautifulSoup
import pandas as pd

def parse_old_reddit_html(path):
    """
    Parse a saved old.reddit HTML page and extract author + comment text.
    Works best when you expanded comments before saving.
    """
    with open(path, "r", encoding="utf-8") as f:
        html = f.read()

    soup = BeautifulSoup(html, "html.parser")

    comments = []

    # Typical old.reddit comment container
    candidates = soup.select('div.thing[data-author]')
    if not candidates:
        candidates = soup.find_all("div", attrs={"data-author": True})

    print("Found candidate elements:", len(candidates))

    for thing in candidates:
        author = thing.get("data-author")

        if not author or author in ["[deleted]", "[removed]"]:
            continue

        # Try to get the markdown body of the comment
        md = thing.select_one("div.usertext-body div.md")
        if md is None:
            md = thing.select_one("div.md")

        text = md.get_text(" ", strip=True) if md else ""

        if text.strip():
            comments.append({
                "author": author,
                "text": text
            })

    return pd.DataFrame(comments)


In [6]:
FILENAME = "/content/AITA for getting my fiancée a “fake” diamond ring_ _ AmItheAsshole.html"  # Corrected filename

df = parse_old_reddit_html(FILENAME)
print("Total comments extracted:", len(df))
df.head()

Found candidate elements: 160
Total comments extracted: 160


,author,text
0,AITAFakeDiamondButNo,I really don’t know what to say. I’m at a loss...
1,TaraBells,Even lab created diamonds (vs CZ$ are generall...
2,ThatB0yAintR1ght,"Yeah, maybe he bought a ring with Moissanite (..."
3,This_Isnt_Progress,My moissanite ring (which I specifically asked...
4,mannequinlolita,Yes a moissanite is Less expensive. Not Inexpe...


In [8]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [9]:
df.to_csv("/content/drive/MyDrive/for getting my fiance a fake diamond ring.csv", index=False)